In [93]:
import numpy as np
import scipy as sp
import scipy.signal 
import cv2
import os
import matplotlib.pyplot as plt
import imageio
MIN_DEPTH = 4

In [94]:
linear = np.array([[0.250], [0.50], [0.250]])
binomial = np.array([[0.0625], [0.2500],[0.3750], [0.2500], [0.0625]])
cubic_01 =  np.array([[-0.0625], [0.0000], [0.3125],[0.5000], [0.3125], [0.0000], [-0.0625]])
cubic_05 = np.array([ [-0.03125], [0.0000],[0.28125],[0.5000], [0.28125], [0.0000], [-0.03125]])
windsinc =  np.array([[0.0000],[-0.0153],[0.0000],[0.2684],[0.4939], [0.2684], [0.0000], [-0.0153], [0.0000]])
qmf_9 =  np.array([[0.0198],[-0.0431],[-0.0519],[0.2932],[0.5638], [0.2932], [-0.0519], [-0.0431] , [0.0198]])
jpeg =  np.array([[0.0267],[-0.0169],[-0.0782],[0.2669],[0.6029], [0.2669], [-0.0782], [-0.0169], [0.0267]]) 

In [95]:
def generatingKernel(kernel):
#     kernel = np.array([0.25-a/2.0, 0.25,a, 0.25,0.25-a/2.0])
    return np.outer(kernel,kernel)

In [96]:
def reduce(image, kernel):
    
    p = int(kernel.shape[0]/2)
    img_h, img_w, C = image.shape
    #padding
    img_pad = cv2.copyMakeBorder(image,p,p,p,p, borderType = cv2.BORDER_REFLECT_101)
    # convolution
    img_conv =  scipy.signal.convolve(img_pad, kernel[:, :, np.newaxis], mode='valid')

    #subsampling
    img_sub = img_conv[::2,::2]
    return img_sub

In [97]:
def expand(image, kernel):
    img_up = image.copy()
    
    for r in range(1,image.shape[0]*2,2):
        img_up = np.insert(img_up,r,0,axis=0)
    for c in range(1,image.shape[1]*2,2):
        img_up = np.insert(img_up,c,0,axis=1)
    
    #padding
    p = int(kernel.shape[0]/2)
    img_pad = cv2.copyMakeBorder(img_up,p,p,p,p, borderType=cv2.BORDER_REFLECT_101)
    #convolution
    img_conv =  scipy.signal.convolve(img_pad, kernel[:, :, np.newaxis], mode='valid')
    # Brightness correction
    img_final = img_conv*4
    return img_final 

In [98]:
def normalize(img):
    return cv2.normalize(img,None,alpha=0,beta=255, norm_type = cv2.NORM_MINMAX)

In [99]:
def downsample(image,kernel,levels):
    kernel = generatingKernel(kernel)
    downsampled = [image.astype(np.float)]
    for level in range(levels):
        new_layer = reduce(downsampled[level],kernel).astype(np.float)
        new_layer = normalize(new_layer)
        downsampled.append(new_layer)
        
    return downsampled

def upsample(img, kernel, levels):
    kernel = generatingKernel(kernel)
    upsampled = [img.astype(np.float)]
    for level in range(levels):
        new_layer = expand(upsampled[level],kernel).astype(np.float)
        new_layer = normalize(new_layer)
        upsampled.append(new_layer)
        
    return  upsampled

In [100]:
image = cv2.imread('cycles.jpg')
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)


In [101]:
upsampled_images = []
liner_images = upsample(image, linear, 4)
binomial_images = upsample(image, binomial, 4)
cubic_05_images = upsample(image, cubic_05, 4)
windsinc_images = upsample(image, windsinc, 4)

In [107]:
upsampled_images = []
for i in range(2):
    upsampled_images.append(liner_images[i].astype(np.uint8))
    upsampled_images.append(np.zeros((50,50),  dtype=np.uint8))
    upsampled_images.append(binomial_images[i].astype(np.uint8))
    upsampled_images.append(np.zeros((50,50), dtype=np.uint8))
    upsampled_images.append(cubic_05_images[i].astype(np.uint8))
    upsampled_images.append(np.zeros((50,50), dtype=np.uint8))
    upsampled_images.append(cubic_05_images[i].astype(np.uint8))
    

In [108]:
imageio.mimsave('./movieup.gif', upsampled_images, fps = 1)

In [109]:
liner_images = downsample(image, linear, 4)
binomial_images = downsample(image, binomial, 4)
cubic_05_images = downsample(image, cubic_05, 4)
windsinc_images = downsample(image, windsinc, 4)


In [110]:
downsampled_images = []
for i in range(4):
    downsampled_images.append(liner_images[i].astype(np.uint8))
    downsampled_images.append(np.zeros((50,50),  dtype=np.uint8))
    downsampled_images.append(binomial_images[i].astype(np.uint8))
    downsampled_images.append(np.zeros((50,50), dtype=np.uint8))
    downsampled_images.append(cubic_05_images[i].astype(np.uint8))
    downsampled_images.append(np.zeros((50,50), dtype=np.uint8))
    downsampled_images.append(cubic_05_images[i].astype(np.uint8))
    

In [111]:
imageio.mimsave('./moviedown.gif', downsampled_images, fps = 1)